In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec

"""wd_5_bigru_cnn

在论文 Recurrent Convolutional Neural Networks for Text Classification 中。

使用 BiRNN 处理，将每个时刻的隐藏状态和原输入拼起来，在进行 max_pooling 操作。

这里有些不同，首先也是使用 bigru 得到每个时刻的，将每个时刻的隐藏状态和原输入拼起来；

然后使用输入到 TextCNN 网络中。

"""





class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = [2, 3, 4, 5, 7]
        self.n_filter = 256
        self.fc_hidden_size = 512
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=5000
        self.lr=0.01
        self.decay_step=7000
        self.decay_rate=0.8
        self.rnn_hidden_units=100






class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.rnn_hidden_units=settings.rnn_hidden_units
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        # placeholders
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')
            
            self.keep_prob_embed = tf.placeholder(name='keep_prob', dtype=tf.float32)
            self.keep_prob_fully = tf.placeholder(name='keep_prob', dtype=tf.float32)
            self.keep_prob_ae = tf.placeholder(name='keep_prob', dtype=tf.float32)
            self.bn_training = tf.placeholder(name='bn_training', dtype=tf.bool)

        p = self.dropout(self.embedded_x_query, self.keep_prob_embed)
        h = self.dropout(self.embedded_x_title, self.keep_prob_embed)
        
        output_title = self.cnn_inference(self.embedded_x_query, self.query_len,'query')
        output_title = tf.expand_dims(output_title, 0)

        output_content = self.cnn_inference(self.embedded_x_title, self.title_len,'title')
        output_content = tf.expand_dims(output_content, 0)
        
        with tf.name_scope("rnn_output"):
            for i in range(3):
                # BiLSTM
                p_state, h_state = p, h
                for j in range(3):
                    with tf.variable_scope(f'p_lstm_{i}_{j}', reuse=None):
                        p_state, _ = self.BiLSTM(tf.concat(p_state, axis=-1))
                    with tf.variable_scope(f'p_lstm_{i}_{j}' + str(i), reuse=None):
                        h_state, _ = self.BiLSTM(tf.concat(h_state, axis=-1))
    
                p_state = tf.concat(p_state, axis=-1)
                h_state = tf.concat(h_state, axis=-1)
                # attention
                cosine = tf.divide(tf.matmul(p_state, tf.matrix_transpose(h_state)),
                                   (tf.norm(p_state, axis=-1, keep_dims=True) * tf.norm(h_state, axis=-1, keep_dims=True)))
                att_matrix = tf.nn.softmax(cosine)
                p_attention = tf.matmul(att_matrix, h_state)
                h_attention = tf.matmul(att_matrix, p_state)
    
                # DesNet
                p = tf.concat((p, p_state, p_attention), axis=-1)
                h = tf.concat((h, h_state, h_attention), axis=-1)
    
                # auto_encoder
                p = tf.layers.dense(p, 200)
                h = tf.layers.dense(h, 200)
    
                p = self.dropout(p, self.keep_prob_ae)
                h = self.dropout(h, self.keep_prob_ae)
    
            # interaction and prediction layer
            add = p + h
            sub = p - h
            norm = tf.norm(sub, axis=-1)
            out = tf.concat((p, h, add, sub, tf.expand_dims(norm, axis=-1)), axis=-1)
            out = tf.reshape(out, shape=(-1, out.shape[1] * out.shape[2]))
            out = self.dropout(out, self.keep_prob_fully)
    
            out = tf.layers.dense(out, 1000, activation='relu')
            out = tf.layers.batch_normalization(out, training=self.bn_training)
            out = tf.layers.dense(out, 1000, activation='relu')
            out = tf.layers.batch_normalization(out, training=self.bn_training)
            out = tf.layers.dense(out, 1000)
            out = tf.layers.batch_normalization(out, training=self.bn_training)
            self.distance = tf.layers.dense(out, 1)

        with tf.variable_scope('fc-bn-layer'):
            output = tf.concat([output_title, output_content], axis=0)
            output = tf.reduce_max(output, axis=0)
            W_fc = self.weight_variable([self.n_filter_total, self.fc_hidden_size], name='Weight_fc')
            h_fc = tf.matmul(output, W_fc, name='h_fc')
            fc_bn = tf.layers.batch_normalization(h_fc, training=self.bn_training)
            self.fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
            fc_bn_drop = tf.nn.dropout(self.fc_bn_relu, self.keep_prob)

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([self.fc_hidden_size, self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self._y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.distance
        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred)



    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)



    def cnn_inference(self, X_inputs, n_step,name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope(("conv-maxpool-%s" % filter_size)+'_'+name):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv_bn=tf.layers.batch_normalization(conv,training=self.bn_training,name=name)
                # conv_bn, update_ema = self.batchnorm(conv, beta, convolutional=True)  # 在激活层前面加 BN
                # Apply nonlinearity, batch norm scaling is not useful with relus
                # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
                h = tf.nn.relu(conv_bn, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
                # self.update_emas.append(update_ema)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]



    def BiLSTM(self, x):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.rnn_hidden_units)
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.rnn_hidden_units)
        return tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, x, dtype=tf.float32)
        
    def dropout(self, x, keep_prob):
        return tf.nn.dropout(x, keep_prob)
        
    









def loader(data_store_path='first_zzp/'):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    print("开始加载数据")
    all_data=pd.read_pickle(data_store_path+"train_data.pickle")
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data,vaild_data

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

def get_w2v_array(word_list,max_len):
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    return array


settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                       settings.decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)
sess = tf.Session() #启动创建的模型
sess.run(tf.initialize_all_variables())


# saver.restore(sess, tf.train.latest_checkpoint('tf_model_rcnn/'))
per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
parts_time=[]
print("start")
for batch in batches:
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)

    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
            # self.keep_prob_embed = tf.placeholder(name='keep_prob', dtype=tf.float32)
            # self.keep_prob_fully = tf.placeholder(name='keep_prob', dtype=tf.float32)
            # self.keep_prob_ae = tf.placeholder(name='keep_prob', dtype=tf.float32)
            # self.bn_training = tf.placeholder(name='bn_training', dtype=tf.bool)
        
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
                model.keep_prob_embed:0.8,
                model.keep_prob_fully:0.8,
                model.keep_prob_ae:0.5,
                model.bn_training:True,
    }
    
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%50==0:
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)


        # #########################这是连续型特征
        # vaild_x_ctr_fea=vaild_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
        # vaild_x_ctr_fea=np.delete(vaild_x_ctr_fea, -1, axis=1)######需要把类别型特征去掉
        # ##########################
        # ###########################这是类别型特征
        # vaild_x_tfidf=vaild_x[:,-1][:,None]
        # #########################################


        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,
                    model.keep_prob_embed:1,
                    model.keep_prob_fully:1,
                    model.keep_prob_ae:1,
                    model.bn_training:False,
        }
        
        
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3)
        saver = saver.save(sess, "/home/kesci/work/td_model_drcn", global_step=step)

开始加载数据
加载数据完成
train starting


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use `tf.global_variables_initializer` instead.
start
step 1,loss:1.24884
step 2,loss:7.94893
step 3,loss:11.1769
step 4,loss:7.45909
step 5,loss:3.62681
step 6,loss:2.00468
step 7,loss:2.09596
step 8,loss:2.82304
step 9,loss:3.84512
step 10,loss:3.55188
step 11,loss:3.75298
step 12,loss:3.15512
step 13,loss:2.67905
step 14,loss:2.52603
step 15,loss:2.43207
step 16,loss:2.50855
step 17,loss:2.22003
step 18,loss:1.85318
step 19,loss:1.73492
step 20,loss:1.65507
step 21,loss:1.47723
step 22,loss:1.75411
step 23,loss:1.37164
step 24,loss:1.04718
step 25,loss:1.77719
step 26,loss:1.13742
step 27,loss:0.795003
step 28,loss:0.735636
step 29,loss:0.674342
step 30,loss

KeyboardInterrupt: 

In [2]:
parts_time=[]
from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
        
                model.keep_prob_embed:0.8,
                model.keep_prob_fully:0.8,
                model.keep_prob_ae:0.5,
                model.bn_training:True,
        
    }

    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,

                    model.keep_prob_embed:1,
                    model.keep_prob_fully:1,
                    model.keep_prob_ae:1,
                    model.bn_training:False,
            
            
        }
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3,var_list=tf.global_variables())
        saver = saver.save(sess, "/home/kesci/work/first_zzp/tf_model_hrcn_textcnn/rnn", global_step=step)

step 120,loss:0.472395
step 130,loss:0.463998
step 140,loss:0.506809
step 150,loss:0.496877
step 160,loss:0.480746
step 170,loss:0.499878
step 180,loss:0.521097
step 190,loss:0.475982
step 200,loss:0.467037
step 210,loss:0.453355
step 220,loss:0.444587
step 230,loss:0.466765
step 240,loss:0.493508
step 250,loss:0.514931
step 260,loss:0.51912
step 270,loss:0.467393
step 280,loss:0.44461
step 290,loss:0.503186
step 300,loss:0.500033
step 310,loss:0.488276
step 320,loss:0.490603
step 330,loss:0.496466
step 340,loss:0.501055
step 350,loss:0.486557
step 360,loss:0.476719
step 370,loss:0.484332
step 380,loss:0.485746
step 390,loss:0.501963
step 400,loss:0.469364
step 410,loss:0.436967
step 420,loss:0.55591
step 430,loss:0.471293
step 440,loss:0.504407
step 450,loss:0.608281
step 460,loss:0.557874
step 470,loss:0.516888
step 480,loss:0.484384
step 490,loss:0.483864
step 500,loss:0.474109
距离训练完一整轮剩余时间： 767.2611294154078  分钟
per_train_loss_avg: 0.48840725056843665 **********
vaild_loss: 0.58266